## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-16-57-19 +0000,nyt,"Trump Plans to Pardon Henry Cuellar, Texas Dem...",https://www.nytimes.com/2025/12/03/us/politics...
1,2025-12-03-16-56-15 +0000,nyt,"Private Employers Cut Jobs in November, in Lat...",https://www.nytimes.com/2025/12/03/business/jo...
2,2025-12-03-16-54-00 +0000,wsj,Fortunes Shift as Airbus Stumbles and Boeing R...,https://www.wsj.com/business/airlines/airbus-c...
3,2025-12-03-16-52-34 +0000,nyt,Storm Floodwaters in Indonesia Turned Logs Int...,https://www.nytimes.com/2025/12/03/world/asia/...
4,2025-12-03-16-48-29 +0000,bbc,Supermarket loyalty discounts allowed on baby ...,https://www.bbc.com/news/articles/clyx321ded1o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,66
183,new,22
69,he,16
75,up,14
281,ukraine,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
274,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,196
232,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...,142
238,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,128
293,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...,119
226,2025-12-03-02-19-03 +0000,cbc,Trump says National Guard will soon go to New ...,https://www.cbc.ca/news/world/trump-new-orlean...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
274,196,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
232,76,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...
272,61,2025-12-02-23-45-31 +0000,latimes,"Baby, baby, baby, baby, oh, baby: New gorilla ...",https://www.latimes.com/california/story/2025-...
293,53,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...
216,53,2025-12-03-03-14-50 +0000,nypost,Marco Rubio claims ‘progress’ towards Ukraine-...,https://nypost.com/2025/12/02/us-news/marco-ru...
217,50,2025-12-03-03-10-40 +0000,bbc,YouTube says it will be less safe for kids und...,https://www.bbc.com/news/articles/cp341gz24p5o...
9,44,2025-12-03-16-39-00 +0000,wsj,EU Launches $3.5 Billion Plan to Secure Raw-Ma...,https://www.wsj.com/world/europe/eu-launches-3...
363,44,2025-12-02-19-15-09 +0000,nypost,"Kraft, Coca-Cola among companies sued by San F...",https://nypost.com/2025/12/02/business/kraft-c...
126,44,2025-12-03-11-41-00 +0000,wsj,House Republicans held on to a congressional d...,https://www.wsj.com/politics/elections/republi...
136,38,2025-12-03-11-00-00 +0000,nypost,Sen. Ron Johnson tells ‘Pod Force One’ he was ...,https://nypost.com/2025/12/03/us-news/sen-ron-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
